This code comes from the textbook example (https://deepnote.com/workspace/hao-pham-efb6-9e45732a-afda-453c-8485-472c524bf503/project/1-Introduction-Duplicate-7231f625-f3dc-4607-8336-24da84aacfc0/notebook/intro-58422166dd0b4e17bb10cf8ad3d886e4)

No significant changes.

# WIP Commanding Robot with NN

TODO: need to debug with anthony

In [4]:
test_inputs = [RigidTransform(
  R=RotationMatrix([
    [-2.3403549959129465e-06, 0.05032888698822864, -0.9987326985380272],
    [0.9999999999051579, -1.3437349077360507e-05, -3.0204696645260106e-06],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[0.4686663152509269, -1.0643346551004039e-05, 0.6012858601360346],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 0.0, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.6427876096865393, 0.0, -0.766044443118978],
    [0.766044443118978, 0.0, 0.6427876096865393],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, -0.06427876096865393, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.984807753012208, 0.0, -0.17364817766693041],
    [0.17364817766693041, 0.0, 0.984807753012208],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, -0.0984807753012208, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.8660254037844387, 0.0, 0.4999999999999998],
    [-0.4999999999999998, 0.0, 0.8660254037844387],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.4, -0.08660254037844388, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.3420201433256689, 0.0, 0.9396926207859083],
    [-0.9396926207859083, 0.0, 0.3420201433256689],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, -0.03420201433256689, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.34202014332566866, 0.0, 0.9396926207859084],
    [-0.9396926207859084, 0.0, -0.34202014332566866],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, 0.034202014332566866, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.8660254037844384, 0.0, 0.5000000000000004],
    [-0.5000000000000004, 0.0, -0.8660254037844384],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.39999999999999997, 0.08660254037844384, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.9848077530122081, 0.0, -0.17364817766692997],
    [0.17364817766692997, 0.0, -0.9848077530122081],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, 0.09848077530122082, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.6427876096865396, 0.0, -0.7660444431189778],
    [0.7660444431189778, 0.0, -0.6427876096865396],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, 0.06427876096865397, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
),
  RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
)]


In [12]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation, AbstractValue)

from pydrake.all import (Box, SpatialInertia, UnitInertia, CoulombFriction, 
                         LeafSystem, ConstantVectorSource)

import torch
from torch import nn

# Start the visualizer.
# meshcat = StartMeshcat()

def BuildAndSimulate():

    builder = DiagramBuilder()
    time_step = .05
    station = builder.AddSystem(
        MakeManipulationStation(filename=FindResource("models/iiwa_and_wsg.dmd.yaml"), 
                                    time_step=time_step))
    # station = builder.AddSystem(
    #     MakeManipulationStation(model_directives, time_step=time_step))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(wsg_position.get_output_port(),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant_robot.GetMyContextFromRoot(context)
    station_context = station.GetMyContextFromRoot(context)
    diff_ik_context = differential_ik.GetMyContextFromRoot(context)

    # provide initial states
    q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                    -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
    # set the joint positions of the kuka arm
    plant_robot.SetPositions(plant_context, iiwa, q0)
    # set rate of simulation
    simulator.set_target_realtime_rate(1.0)

    # NN
    # time_step is defined above on line 25 to be 1.0
    model = torch.nn.Linear(3,3)
    for t in range(0, 20):
        print("time", t*time_step)
        # pass in new input to NN
        # get prediction

        # get gripper xyz position
        gripper_ind = plant_robot.GetBodyIndices(iiwa)[-1]
        gripper = plant_robot.get_body(gripper_ind)
        # X_WG_init = gripper.EvalPoseInWorld(plant_context)
        # X_WG_next = RigidTransform(
        #                 X_WG_init.rotation(), 
        #                 X_WG_init.translation() + np.array([.0001, 0, 0]))
        # print('X_WG_init', X_WG_init)

        # test following trajectory
        X_WG_next = test_inputs[t]
        print('gripper_next', X_WG_next.translation())
        differential_ik.get_input_port(0).FixValue(
                    diff_ik_context, 
                    X_WG_next)
        # view the change in simulation
        simulator.AdvanceTo(t*time_step)
        # display the joint angles
        print("iiwa angles meas", station.GetOutputPort("iiwa_position_measured").Eval(station_context))
        # display the pose
        print("iiwa pose meas",gripper.EvalPoseInWorld(plant_context).translation())
        

BuildAndSimulate()

time 0.0
gripper_next [ 4.68666315e-01 -1.06433466e-05  6.01285860e-01]
iiwa angles meas [ 1.40697944e-05  1.56461162e-01 -3.82982456e-05 -1.32296976e+00
 -6.29924852e-06  1.61181158e+00 -2.67085450e-05]
iiwa pose meas [ 4.64136714e-01 -9.44211721e-06  6.91171803e-01]
time 0.05
gripper_next [0.55 0.   0.4 ]
iiwa angles meas [ 1.40697944e-05  1.56461162e-01 -3.82982456e-05 -1.32296976e+00
 -6.29924852e-06  1.61181158e+00 -2.67085450e-05]
iiwa pose meas [ 4.64136714e-01 -9.44211721e-06  6.91171803e-01]
time 0.1
gripper_next [ 0.52660444 -0.06427876  0.4       ]
iiwa angles meas [ 1.09959411e-03  1.56641884e-01 -5.14800442e-04 -1.33922465e+00
 -4.00928102e-03  1.59959463e+00  4.43210939e-03]
iiwa pose meas [ 4.64414175e-01 -2.04640787e-05  6.84601582e-01]
time 0.15000000000000002
gripper_next [ 0.46736482 -0.09848078  0.4       ]
iiwa angles meas [-0.00319652  0.17496025 -0.00488443 -1.42276441 -0.02421239  1.52478497
  0.02244094]
iiwa pose meas [ 0.47040322 -0.00545628  0.64268518]
time

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>